Install Required Libraries

In [1]:
%pip install transformers torch tensorflow scikit-learn nltk networkx matplotlib pandas PyPDF2

Note: you may need to restart the kernel to use updated packages.


Import Libraries

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import ast
import matplotlib.pyplot as plt
import networkx as nx
import PyPDF2
import pickle

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chand_xiu5rxo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Text Cleaning Functions

In [3]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def remove_stopwords(text):
    words = text.split()
    filtered = [w for w in words if w not in stop_words]
    return " ".join(filtered)

PDF Text Extractor

In [4]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + " "
    return text

Experience Extractor

In [5]:
def extract_experience(text):
    text = text.lower()

    patterns = [
        r'(\d+)\+?\s*years',
        r'(\d+)\+?\s*yrs',
        r'(\d+)\+?\s*year'
    ]

    experience_years = []

    for pattern in patterns:
        matches = re.findall(pattern, text)
        for match in matches:
            experience_years.append(int(match))

    if experience_years:
        return max(experience_years)
    else:
        return "Not Found"

Load Dataset & Train Model

In [6]:
df = pd.read_csv("all_job_post.csv")

df["job_description"] = df["job_description"].apply(clean_text).apply(remove_stopwords)
df["skills"] = df["job_skill_set"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

texts = df["job_description"].tolist()
labels = df["skills"].tolist()

tokenizer_lstm = Tokenizer(num_words=10000)
tokenizer_lstm.fit_on_texts(texts)

sequences = tokenizer_lstm.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=150)

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
for cat in df["category"].unique():
    print(cat)

HR
INFORMATION-TECHNOLOGY
SALES
FINANCE
BUSINESS-DEVELOPMENT


BiLSTM Model

In [8]:
model = Sequential()
model.add(Input(shape=(150,)))
model.add(Embedding(input_dim=10000, output_dim=128))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dense(y.shape[1], activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[tf.keras.metrics.AUC(name="auc")]
)

model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - auc: 0.5333 - loss: 0.4973 - val_auc: 0.6796 - val_loss: 0.0206
Epoch 2/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 117ms/step - auc: 0.6947 - loss: 0.0197 - val_auc: 0.7386 - val_loss: 0.0202
Epoch 3/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 175ms/step - auc: 0.7705 - loss: 0.0179 - val_auc: 0.7784 - val_loss: 0.0179
Epoch 4/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 147ms/step - auc: 0.7978 - loss: 0.0162 - val_auc: 0.7877 - val_loss: 0.0173
Epoch 5/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - auc: 0.7991 - loss: 0.0158 - val_auc: 0.7845 - val_loss: 0.0172
Epoch 6/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 160ms/step - auc: 0.7949 - loss: 0.0158 - val_auc: 0.7820 - val_loss: 0.0173
Epoch 7/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - auc: 0.7947 - loss: 0.0158 - val_auc: 0.7832 - val_loss: 0.0174
Epoch 8/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 137ms/step - auc: 0.7942 - loss: 0.0158 - val_auc: 0.7826 - val_loss: 0.0175
Epoch 9/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 157ms/step - a

Prediction Function

In [9]:
def predict_skills_bilstm(job_text, threshold=0.4):
    job_text = clean_text(job_text)
    job_text = remove_stopwords(job_text)

    seq = tokenizer_lstm.texts_to_sequences([job_text])
    X_input = pad_sequences(seq, maxlen=150)

    pred = model.predict(X_input)[0]
    predicted_skills = [mlb.classes_[i] for i, val in enumerate(pred) if val > threshold]

    return predicted_skills

Master Skill Dictionary

In [10]:
MASTER_SKILLS = {

    # Programming Languages
    "python","java","c++","c#","javascript","typescript","go","rust","scala","kotlin",
    "php","ruby","swift","r","matlab","perl","bash","powershell",

    # Web Development
    "html","css","react","angular","vue","next.js","node.js","express","django","flask",
    "spring","spring boot","laravel","asp.net","jquery","bootstrap","tailwind",

    # Databases
    "sql","mysql","postgresql","oracle","mongodb","redis","cassandra","dynamodb",
    "firebase","sqlite","neo4j","mariadb","elasticsearch",

    # Cloud Platforms
    "aws","azure","gcp","google cloud","amazon web services","microsoft azure",
    "ec2","s3","lambda","cloudformation","iam","rds","eks",
    "azure data factory","adf","azure devops","azure functions","blob storage",
    "bigquery","cloud run","cloud functions",

    # DevOps Tools
    "docker","kubernetes","terraform","ansible","jenkins","gitlab","github actions",
    "ci/cd","devops","helm","prometheus","grafana","nagios",

    # Data Engineering
    "etl","elt","data pipeline","data pipelines","airflow","apache airflow",
    "spark","pyspark","hadoop","hive","pig","kafka","databricks","snowflake",
    "redshift","synapse","delta lake",

    # Machine Learning / AI
    "machine learning","deep learning","nlp","computer vision","pytorch","tensorflow",
    "keras","scikit-learn","xgboost","opencv","huggingface","bert","llm",
    "langchain","rag","genai","generative ai",

    # Cybersecurity
    "cyber security","penetration testing","ethical hacking","network security",
    "cryptography","siem","splunk","firewall","ids","ips","vulnerability assessment",

    # Mobile Development
    "android","ios","flutter","react native","swiftui","kotlin android",

    # Testing / QA
    "selenium","cypress","junit","pytest","automation testing","manual testing",
    "testng","postman","api testing",

    # Tools / Version Control
    "git","github","bitbucket","jira","confluence","linux","unix","windows",
    "visual studio","vscode","intellij","eclipse",

    # Networking
    "tcp/ip","dns","http","https","vpn","routing","switching",

    # Analytics
    "power bi","tableau","excel","data analysis","data visualization","looker",

    # Others
    "microservices","rest api","graphql","soap","system design","distributed systems",
    "oop","data structures","algorithms"
}


Rule-Based Extractor

In [11]:
def universal_skill_extractor(text):
    text = text.lower()
    found_skills = [skill for skill in MASTER_SKILLS if skill in text]
    return sorted(list(set(found_skills)))

Final Skill Extractor

In [12]:
def final_skill_extractor(job_desc):
    bilstm_skills = predict_skills_bilstm(job_desc, threshold=0.4)
    rule_skills = universal_skill_extractor(job_desc)
    final_skills = list(set(bilstm_skills + rule_skills))
    return sorted(final_skills)

In [13]:
test_jd = """
Looking for a Data Engineer with strong Python, SQL, ETL, Azure Data Factory, Spark, Databricks and data pipelines experience.
"""

skills_found = final_skill_extractor(test_jd)

print("\n✅ Extracted Skills:")
print(skills_found)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

✅ Extracted Skills:
['azure', 'azure data factory', 'communication', 'data pipeline', 'data pipelines', 'databricks', 'etl', 'problem solving', 'python', 'r', 'spark', 'sql']


Resume Input + Full Extraction

In [14]:
pdf_path = input("Enter Resume PDF Path: ")

resume_text = extract_text_from_pdf(pdf_path)

skills_found = final_skill_extractor(resume_text)
experience_years = extract_experience(resume_text)

print("\n✅ Extracted Skills:")
print(skills_found)

print("\n📊 Total Experience (Years):")
print(experience_years)

Enter Resume PDF Path:  C:\Users\chand_xiu5rxo\Downloads\Sales-Executive-Resume.pdf


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

✅ Extracted Skills:
['communication', 'excel', 'go', 'ips', 'problem solving', 'r', 'rag']

📊 Total Experience (Years):
20


Save the model 

In [15]:
import pickle
import os

# Create folder to store model files
os.makedirs("saved_model", exist_ok=True)

# 1️⃣ Save BiLSTM Model
model.save("saved_model/bilstm_skill_model.h5")

# 2️⃣ Save Tokenizer
with open("saved_model/tokenizer_lstm.pkl", "wb") as f:
    pickle.dump(tokenizer_lstm, f)

# 3️⃣ Save MultiLabelBinarizer
with open("saved_model/mlb.pkl", "wb") as f:
    pickle.dump(mlb, f)

# 4️⃣ Save sequence length (IMPORTANT)
with open("saved_model/config.pkl", "wb") as f:
    pickle.dump({"maxlen":150}, f)

print("✅ Model, Tokenizer, MLB and Config saved successfully!")

✅ Model, Tokenizer, MLB and Config saved successfully!
